In [1]:
%run "..\src\utils\style_and_packages.ipynb"

# Beispiel: Balken mit Tilger{#sec-tilger}


## Aufgabenstellung

Das Beispiel ist die Weiterführung der Aufgabe in @sec-ems_untilg.
Ein einfacher Balken mit einer Einzelmasse, welcher in dieser Aufgabe mit Tilger ausgestattet ist, ist in @fig-mms_tilg_tilger dargestellt. Die Masse erfährt eine dynamische Einwirkung durch die Funktion $F(t)$. 

![Statisches System des Balkens mit Tilger](../images/aufgabe_mms_tilg_system.svg){#fig-mms_tilg_tilger}

Gesucht:

- Normierte Eigenform
- Maximale dynamische Verformung mittels stationärer Lösung
- Maximale dynamische Beschleunigung mittels stationärer Lösung

Gegeben:






In [2]:
E, I, m_H, m_T, k_T, L, F_0, omega, delta_11, delta_12 = sp.symbols('E, I, m_H, m_T, k_T, L, F_0, omega, delta_11, delta_12')

t = sp.symbols('t')

k_T, k_H = sp.symbols('k_T, k_H')

omega_n =sp.symbols('omega_n')

phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')


In [3]:
#| label: tbl-parameter_mms4
#| tbl-cap: Parameter der Aufgabenstellung

params = {E: 200*10**3 *unit.N/unit.mm**2,
          I:2*10**8*unit.mm**4,
          m_H:2000*unit.N*unit.second**2/unit.m,
          m_T:150*unit.N*unit.second**2/unit.m,
          k_T:90*10**3*unit.N/unit.m,
          L:5*unit.m,
          F_0:0.8*10**3*unit.N,
          omega:12.6/unit.second,
          phi_11:1,
          phi_12:1,
          zeta:0.0         
          }

render.dict_to_table(params)

|   |   |
|---|---|
| $E = \frac{200000 \text{N}}{\text{mm}^{2}}$ | $F_{0} = 800.0 \text{N}$ |
| $I = 200000000 \text{mm}^{4}$ | $L = 5 \text{m}$ |
| $k_{T} = \frac{90000 \text{N}}{\text{m}}$ | $m_{H} = \frac{2000 \text{N} \text{s}^{2}}{\text{m}}$ |
| $m_{T} = \frac{150 \text{N} \text{s}^{2}}{\text{m}}$ | $\omega = \frac{12.6}{\text{s}}$ |
| $\phi_{11} = 1$ | $\phi_{12} = 1$ |
| $\zeta = 0.0$ | |



$$
F(t) = F_0 \cdot \sin(\omega\cdot t) = 0.8 \text{kN} \cdot (12.6 \frac{\text{rad}}{\text{s}}\cdot t)
$$

{{< pagebreak >}}

## Musterlösung

In [4]:
F_t = F_0 * sp.sin(omega*t)

### Bemerkung Tilgerauslegung

Die Auslegung des Tilgers kann folgender massen geschehen:

- Tilgermasse $5\%$ von der Masse des Hauptträgers.
- Optimale Frequenz bestimmen: 
$$
f_{T,opt} =\frac{f_H}{1+\frac{m_T}{m_H}}
$${#eq-mms_tilg_optimale_frequenz}
- Daraus die optimale Steifigkeit bestimmen:
$$
k_{T,opt} = (2 \pi f_{T,opt})^2
$${#eq-mms_tilg_optimale_steif}


### Steifigkeitsmatrix $\mathbf{K}$


![Auslenkung am ersten Freiheitsgrad](../images/aufgabe_mms_tilg_auslenk2.svg){#fig-mms_tilg_auslenkung_1}


Wichtig dabei sind die Richtungen der Kräfte. Als Denkstütze gilt folgendes:

- Der Auslenkung um $u$ wirkt die Federkraft entgegen, welche $k u$ entspricht.
- Zusätzlich wirkt die Trägheitskraft der Auslenkung entgegen, welche $m u''$ entspricht.
- Nach der Betrachtung des ausgelenkten Punkts, kann mittels *Actio-Reactio*-Prinzip das "*Stockwerk*" ins Gleichgewicht gebracht werden.
- Vorzeichen sind gegen der Bewegungsrichtig positiv.

![Auslenkung am zweiten Freiheitsgrad](../images/aufgabe_mms_tilg_auslenk1.svg){#fig-mms_tilg_auslenkung_2}


In [5]:

params['k_H'] = (48 * E*I / (2*L)**3).subs(params).simplify()
K = sp.Matrix([[k_H + k_T, -k_T],[-k_T, k_T]])


render.eq_display(sp.MatrixSymbol('K', 2,2), K,
                  sp.MatrixSymbol('K', 2,2), K.subs(params),
                  )

    ⎡k_H + k_T  -k_T⎤
K = ⎢               ⎥
    ⎣  -k_T     k_T ⎦

    ⎡2010000⋅newton  -90000⋅newton ⎤
    ⎢──────────────  ──────────────⎥
    ⎢    meter           meter     ⎥
K = ⎢                              ⎥
    ⎢-90000⋅newton    90000⋅newton ⎥
    ⎢──────────────   ──────────── ⎥
    ⎣    meter           meter     ⎦

### Eigenvektoren
#### Massenmatrix $\mathbf{M}$ 

Die Massenmatrix folgt dem Aufbau der Steifigkeitsmatrix. Es gelten die gleichen Vorzeichenregelungen. Die Einträge beziehen sich auf @fig-mms_tilg_auslenkung_2 und @fig-mms_tilg_auslenkung_1.

In [6]:
M = sp.Matrix([[m_H, 0],[0, m_T]])

render.eq_display(sp.MatrixSymbol('M', 2,2), M,
                  sp.MatrixSymbol('M', 2,2), M.subs(params))

    ⎡m_H   0 ⎤
M = ⎢        ⎥
    ⎣ 0   m_T⎦

    ⎡                  2                    ⎤
    ⎢2000⋅newton⋅second                     ⎥
    ⎢───────────────────          0         ⎥
    ⎢       meter                           ⎥
M = ⎢                                       ⎥
    ⎢                                      2⎥
    ⎢                     150⋅newton⋅second ⎥
    ⎢         0           ──────────────────⎥
    ⎣                           meter       ⎦

#### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand folgender Gleichung bestimmen:

$$
\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}
$${#eq-mms_tilg_eigenkreisfrequenzen}

In [7]:
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

      23.3 
ω₁ = ──────
     second

      32.6 
ω₂ = ──────
     second

#### Eigenvektoren $\phi$

$$
\phi_n = \begin{bmatrix}
\phi_{1n}\\
\phi_{2n} 
\end{bmatrix}
$${#eq-mms_tilg_eigenvek}
$$
[\mathbf{K}-\omega_n^2 \mathbf{M}]\cdot \begin{bmatrix}
\phi_{1n}\\
\phi_{2n} 
\end{bmatrix}
=0$${#eq-mms_tilg_eigenvek_eq}


Dazu ist die entsprechende Normierung aus der Aufgabenstellung zu berücksichtigen.

In [8]:

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_21 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_21).values())[0]

params['phi_21'] = phi_21.subs(params).simplify() 




render.eq_display(sp.simplify((K-omega_1**2 *M)*phi_1),sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))


⎡                   ⎛                                _________________________
⎢                   ⎜                               ╱    2    2               
⎢               φ₁₁⋅⎝-k_T⋅m_H + m_T⋅(k_H + k_T) + ╲╱  k_H ⋅m_T  - 2⋅k_H⋅k_T⋅m_
⎢-k_T⋅m_T⋅φ₂₁ + ──────────────────────────────────────────────────────────────
⎢                                                                             
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                     m_T     
⎢                                                                             
⎢                   ⎛                               __________________________
⎢                   ⎜                              ╱    2    2                
⎢               φ₂₁⋅⎝k_T⋅m_H - m_T⋅(k_H + k_T) + ╲╱  k_H ⋅m_T  - 2⋅k_H⋅k_T⋅m_H
⎢-k_T⋅m_H⋅φ₁₁ + ──────────────────────────────────────────────────────────────
⎢                                                   

     ⎡1.0 ⎤
φ₁ = ⎢    ⎥
     ⎣10.3⎦

In [9]:
phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_22 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_22).values())[0]

params['phi_22'] = phi_22.subs(params).simplify() 




render.eq_display(sp.simplify((K-omega_2**2 *M)*phi_2),sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))


⎡                   ⎛                                _________________________
⎢                   ⎜                               ╱    2    2               
⎢               φ₁₂⋅⎝-k_T⋅m_H + m_T⋅(k_H + k_T) - ╲╱  k_H ⋅m_T  - 2⋅k_H⋅k_T⋅m_
⎢-k_T⋅m_T⋅φ₂₂ + ──────────────────────────────────────────────────────────────
⎢                                                                             
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                     m_T     
⎢                                                                             
⎢                   ⎛                               __________________________
⎢                   ⎜                              ╱    2    2                
⎢               φ₂₂⋅⎝k_T⋅m_H - m_T⋅(k_H + k_T) - ╲╱  k_H ⋅m_T  - 2⋅k_H⋅k_T⋅m_H
⎢-k_T⋅m_H⋅φ₁₂ + ──────────────────────────────────────────────────────────────
⎢                                                   

     ⎡1.0 ⎤
φ₂ = ⎢    ⎥
     ⎣-1.3⎦

#### Orthogonalitätsbedingung 

Zur Entkoppelung des Systems wird die Orthogonalität der Eigenvektoren kontrolliert. Siehe @sec-mms_nach_ortho für eine ausführliche Erklärung.

In [10]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*M*phi_1)).subs(params).evalf(7),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(7))

           ⎡                     2⎤
  T        ⎢1.79e+4⋅newton⋅second ⎥
φ₁ ⋅M⋅φ₁ = ⎢──────────────────────⎥
           ⎣        meter         ⎦

           ⎡                     2⎤
  T        ⎢2.25e+3⋅newton⋅second ⎥
φ₂ ⋅M⋅φ₂ = ⎢──────────────────────⎥
           ⎣        meter         ⎦

           ⎡                         2⎤
  T        ⎢6.103516e-5⋅newton⋅second ⎥
φ₂ ⋅M⋅φ₁ = ⎢──────────────────────────⎥
           ⎣          meter           ⎦

           ⎡                         2⎤
  T        ⎢6.103516e-5⋅newton⋅second ⎥
φ₁ ⋅M⋅φ₂ = ⎢──────────────────────────⎥
           ⎣          meter           ⎦

Es sind kleine numerische Differenzen zu erkennen welche nicht relevant sind.

In [11]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(3))

  T        ⎡9.7e+6⋅newton⎤
φ₁ ⋅K⋅φ₁ = ⎢─────────────⎥
           ⎣    meter    ⎦

  T        ⎡2.39e+6⋅newton⎤
φ₂ ⋅K⋅φ₂ = ⎢──────────────⎥
           ⎣    meter     ⎦

  T           
φ₂ ⋅K⋅φ₁ = [0]

  T           
φ₁ ⋅K⋅φ₂ = [0]

### Modale Analyse

Die Bewegungsgleichung für einen ungedämpften, periodisch, harmonisch angeregten Mehrmassenschwinger lässt sich folgend beschreiben:

$$
\mathbf{M u''(t) + K u = F(t)}
$${#eq-mms_tilg_bewegungsgleichung_nohomo}

Die Matrix-Gleichung beschreibt ein System aus Differentialgleichungen. Die Modale Analyse zielt darauf ab, diese zu entkoppeln. Bezogen auf den Mehrmassenschwinger heisst eine Entkoppelung, dass diese in Einmassenschwinger aufgeteilt werden. Dies wird nun schrittweise durchgeführt.

#### Modal- und Spektralmatrix

Mittels der Modal- und Spektralmatrix können die generalisierten Grössen ermittelt werden. Diese sind die Eigenschaften der einzelnen Einmassenschwinger. Die generalisierten Werte besitzen keine physikalische Bedeutung, sie sind abhängig von der Wahl der Eigenvektoren, welche bekanntlich von der Normierung abhängen.


Anhand der Bewegungsgleichung können die generalisierten Grössen bestimmt werden, es gilt:

$$\Phi^T M \Phi u''(t) + \Phi^T K \Phi u(t) = \Phi^T F(t)$$

$$M^*u''(t) + K^* u(t) = F^* (t)$$

Alle $N$-Eigenwerte und alle $N$-Eigenvektoren können kompakt
mit Matrizen ausgedrückt werden:


In [12]:
Phi = sp.Matrix([[phi_1, phi_2]])
Omega = sp.Matrix([[omega_1, 0],[0, omega_2]])

render.eq_display('Modalmatrix', 'Phi',
                  sp.MatrixSymbol('Phi', 2, 2), Phi.subs(params).evalf(4),
                  'Spektralmatrix', 'Omega^2',
                  sp.MatrixSymbol('Omega^2', 2, 2), (Omega**2).subs(params).evalf(4))

Modalmatrix = Φ

    ⎡1.0    1.0  ⎤
Φ = ⎢            ⎥
    ⎣10.3  -1.295⎦

                  2
Spektralmatrix = Ω 

     ⎡ 541.7          ⎤
     ⎢───────     0   ⎥
     ⎢      2         ⎥
     ⎢second          ⎥
Ω² = ⎢                ⎥
     ⎢          1063.0⎥
     ⎢   0     ───────⎥
     ⎢               2⎥
     ⎣         second ⎦

#### Generalisierte Grössen


In [13]:
M_star = Phi.T * M * Phi
K_star = Phi.T * K * Phi
F_t_matrix = sp.Matrix([[F_t],[0]])
F_t_star = Phi.T * F_t_matrix

render.eq_display(sp.MatrixSymbol('M^\star', 2,2),sp.simplify(M_star.subs(params)).evalf(5),
                  sp.MatrixSymbol('K^\star', 2,2),sp.simplify(K_star.subs(params)).evalf(5),
                  sp.MatrixSymbol('F(t)', 2,1),sp.simplify(F_t_matrix.subs(params)).evalf(5),
                  sp.MatrixSymbol('F(t)^\star', 2,1),sp.simplify(F_t_star.subs(params)).evalf(5))

           ⎡                     2                       ⎤
           ⎢17898.0⋅newton⋅second                        ⎥
           ⎢──────────────────────            0          ⎥
           ⎢        meter                                ⎥
M__\star = ⎢                                             ⎥
           ⎢                                            2⎥
           ⎢                        2251.6⋅newton⋅second ⎥
           ⎢          0             ─────────────────────⎥
           ⎣                                meter        ⎦

           ⎡9.6959e+6⋅newton                  ⎤
           ⎢────────────────         0        ⎥
           ⎢     meter                        ⎥
K__\star = ⎢                                  ⎥
           ⎢                  2.3941e+6⋅newton⎥
           ⎢       0          ────────────────⎥
           ⎣                       meter      ⎦

       ⎡         ⎛12.6⋅t⎞       ⎤
       ⎢800.0⋅sin⎜──────⎟⋅newton⎥
F(t) = ⎢         ⎝second⎠       ⎥
       ⎢                        ⎥
       ⎣           0            ⎦

              ⎡         ⎛12.6⋅t⎞       ⎤
              ⎢800.0⋅sin⎜──────⎟⋅newton⎥
              ⎢         ⎝second⎠       ⎥
F(t)__\star = ⎢                        ⎥
              ⎢         ⎛12.6⋅t⎞       ⎥
              ⎢800.0⋅sin⎜──────⎟⋅newton⎥
              ⎣         ⎝second⎠       ⎦

#### Kontrolle der modalen Transformation

Die entkoppelten Einmassenschwinger müssen das Verhalten des gesamten Systems abbilden, aus diesem Grund dürfen sich die Eigenkreisfrequenzen nach der Transformation nicht ändern. Dies wird folgend kontrolliert:

In [14]:
omega_1_kontrolle = sp.sqrt(K_star[0] / M_star[0])
omega_2_kontrolle = sp.sqrt(K_star[3] / M_star[3])

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_1_modal', omega_1_kontrolle.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(4),
                  'omega_2_modal', omega_2_kontrolle.subs(params).simplify().evalf(4))




      23.3 
ω₁ = ──────
     second

             23.3 
ω_1_modal = ──────
            second

     32.61 
ω₂ = ──────
     second

            32.61 
ω_2_modal = ──────
            second

#### Modale Höhen

Die modalen Höhen bestimmen sich aus @eq-mms_tilg_modale_hoehe:

$$
H_n = \frac{L_n^\theta}{L_n}
$${#eq-mms_tilg_modale_hoehe}


$$
L_n = \phi_n^T \cdot \mathbf{M 1}
$${#eq-mms_tilg_L_n}

$$
L_n^\theta = \sum_{j=1}^N H_j \cdot m_j \cdot \phi_{jn}
$${#eq-mms_tilg_L_n_theta}

[Wie sind modale Höhen mit diesem Beispiel zu bestimmen?]{.underline}

### Stationäre Antwort

Die stationäre Antwort wird mittels des Vergrösserungsfaktors bestimmt.

Die entkoppelte Differentialgleichung ist nun die folgende:

$$
m^\star_1 q_1''(t) + k^\star_1 q_1(t) = F^\star_1(t) = F^\star_1 \sin(\omega t)
$${#eq-mms_tilger_erste_bewegungsgleichung}

$$
m^\star_2 q_2''(t) + k^\star_2 q_2(t) = F^\star_2(t) = F^\star_2 \sin(\omega t)
$${#eq-mms_tilger_zweite_bewegungsgleichung}

Lösen lässt sich dies mit dem bekannten Ansatz:

$$
q_n(t) = A_n \sin(\omega t) + B_n \cos(\omega)
$${#eq-mms_tilger_ansatzfunktion}

Hier wird jedoch mit dem Vorgehen des Vergrösserungsfaktors verfahren:


#### Verformung

In [15]:
zeta = sp.symbols('zeta')

V_1_omega = 1/(sp.sqrt((1-(omega/omega_1)**2)**2 + (2*zeta*(omega/omega_1))**2))

q_1_stat = F_0 / K_star[0]
q_1_max = q_1_stat*V_1_omega

render.eq_display('V_1(omega)', '1/(sqrt((1-(omega/omega_1)**2)**2 + (2*zeta_*(omega/omega_1))**2))',
                  'V_1(omega)', V_1_omega.subs(params).simplify().evalf(3),
                  'q_1_stat', unit.convert_to(q_1_stat.subs(params).simplify().evalf(3), unit.mm),
                  'q_1_max', 'q_1_stat * V_1(omega)',
                  'q_1_max', unit.convert_to(q_1_max.subs(params).simplify().evalf(3), unit.mm),)

                       1               
V₁(ω) = ───────────────────────────────
               ________________________
              ╱                      2 
             ╱     2  2   ⎛    2    ⎞  
            ╱   4⋅ω ⋅ζ    ⎜   ω     ⎟  
           ╱    ─────── + ⎜- ─── + 1⎟  
          ╱         2     ⎜    2    ⎟  
        ╲╱        ω₁      ⎝  ω₁     ⎠  

V₁(ω) = 1.41

q₁ ₛₜₐₜ = 0.0825⋅millimeter

q₁ ₘₐₓ = q₁ ₛₜₐₜ⋅V₁(ω)

q₁ ₘₐₓ = 0.117⋅millimeter

In [16]:

V_2_omega = 1/(sp.sqrt((1-(omega/omega_2)**2)**2 + (2*zeta*(omega/omega_2))**2))

q_2_stat = F_0 / K_star[3]
q_2_max = q_2_stat*V_2_omega



render.eq_display('V_2(omega)', '1/(sqrt((1-(omega/omega_2)**2)**2 + (2*zeta_*(omega/omega_2))**2))',
                  'V_2(omega)', V_2_omega.subs(params).simplify().evalf(3),
                  'q_2_stat', unit.convert_to(q_2_stat.subs(params).simplify().evalf(3), unit.mm),
                  'q_2_max', 'q_2_stat * V_2(omega)',
                  'q_2_max', unit.convert_to(q_2_max.subs(params).simplify().evalf(3), unit.mm))

                       1               
V₂(ω) = ───────────────────────────────
               ________________________
              ╱                      2 
             ╱     2  2   ⎛    2    ⎞  
            ╱   4⋅ω ⋅ζ    ⎜   ω     ⎟  
           ╱    ─────── + ⎜- ─── + 1⎟  
          ╱         2     ⎜    2    ⎟  
        ╲╱        ω₂      ⎝  ω₂     ⎠  

V₂(ω) = 1.18

q₂ ₛₜₐₜ = 0.334⋅millimeter

q₂ ₘₐₓ = q₂ ₛₜₐₜ⋅V₂(ω)

q₂ ₘₐₓ = 0.393⋅millimeter

##### Effektive Deformation

Die effektiven Grössen resultieren durch Multiplikation mit dem Eigenvektor. Für die erste Eigenkreisfrequenz:

In [17]:
u_1_stat = q_1_max*phi_1
u_2_stat = q_2_max*phi_2
u_stat = u_1_stat + u_2_stat

render.eq_display("u_1_stat","q_1_max*phi_1",
'Matrix([[u_11stat],[u_21stat]])',unit.convert_to(u_1_stat.subs(params).evalf(3),unit.mm))

u₁ ₛₜₐₜ = φ₁⋅q₁ ₘₐₓ

⎡u₁₁ₛₜₐₜ⎤   ⎡0.117⋅millimeter⎤
⎢       ⎥ = ⎢                ⎥
⎣u₂₁ₛₜₐₜ⎦   ⎣ 1.2⋅millimeter ⎦

Sowie für die zweite Eigenkreisfrequenz:

In [18]:
render.eq_display("u_2_stat","q_2_max*phi_2",
'Matrix([[u_12stat],[u_22stat]])',unit.convert_to(u_2_stat.subs(params).evalf(3),unit.mm))

u₂ ₛₜₐₜ = φ₂⋅q₂ ₘₐₓ

⎡u₁₂ₛₜₐₜ⎤   ⎡0.393⋅millimeter ⎤
⎢       ⎥ = ⎢                 ⎥
⎣u₂₂ₛₜₐₜ⎦   ⎣-0.509⋅millimeter⎦

Durch Addition der beiden Verformungen: [Sollte hier die SRSS-Regel verwendet werden?]{.underline}

In [19]:
render.eq_display("u_stat" , "u_1_stat + u_2_stat",
'Matrix([[u_1max],[u_2max]])',unit.convert_to(u_stat.subs(params).evalf(3),unit.mm))


uₛₜₐₜ = u₁ ₛₜₐₜ + u₂ ₛₜₐₜ

⎡u₁ₘₐₓ⎤   ⎡0.51⋅millimeter ⎤
⎢     ⎥ = ⎢                ⎥
⎣u₂ₘₐₓ⎦   ⎣0.693⋅millimeter⎦

#### Beschleunigung

In [20]:
V_a1_omega = omega**2 / omega_1**2 * V_1_omega

q_2_1_max =  F_0 / M_star[0] * V_a1_omega

render.eq_display("V_a1_omega" , "omega**2 / omega_1**2 * V_1_omega",
'V_a1(omega)', V_a1_omega.subs(params).simplify().evalf(3),
                  'Derivative(q_max,t,2)', q_2_1_max.subs(params).simplify().evalf(3))

                2
         V_1_ω⋅ω 
V_a1_ω = ────────
             2   
           ω₁    

Vₐ₁(ω) = 0.415

  2                     
 d          0.0185⋅meter
───(qₘₐₓ) = ────────────
  2                 2   
dt            second    

In [21]:
V_a2_omega = omega**2 / omega_2**2 * V_2_omega

q_2_2_max =  F_0 / M_star[3] * V_a2_omega

render.eq_display('V_a2(omega)', V_a2_omega.subs(params).simplify().evalf(3),'Derivative(q_max,t,2)', q_2_2_max.subs(params).simplify().evalf(3))

Vₐ₂(ω) = 0.176

  2                     
 d          0.0624⋅meter
───(qₘₐₓ) = ────────────
  2                 2   
dt            second    

##### Effektive Beschleunigung

Gleiches Vorgehen wie bei der Deformation.

In [22]:
u_2_1_stat = q_2_1_max*phi_1
u_2_2_stat = q_2_2_max*phi_2
u_2_stat = u_2_1_stat + u_2_2_stat

render.eq_display('Matrix([[Derivative(u_1max,t,2)],[Derivative(u_2max,t,2)]])',u_2_stat.subs(params).evalf(3))


⎡  2       ⎤                 
⎢ d        ⎥   ⎡0.0809⋅meter⎤
⎢───(u₁ₘₐₓ)⎥   ⎢────────────⎥
⎢  2       ⎥   ⎢        2   ⎥
⎢dt        ⎥   ⎢  second    ⎥
⎢          ⎥ = ⎢            ⎥
⎢  2       ⎥   ⎢ 0.11⋅meter ⎥
⎢ d        ⎥   ⎢ ────────── ⎥
⎢───(u₂ₘₐₓ)⎥   ⎢        2   ⎥
⎢  2       ⎥   ⎣  second    ⎦
⎣dt        ⎦                 